In [ ]:
# Autenticar con Earth Engine
#ee.Authenticate()
ee.Initialize()

In [60]:
import ee
import eefolium as geemap
import pandas as pd
import geemap

In [64]:
# Definir los parámetros
collection = "MODIS/061/MCD19A2_GRANULES"
band = "Optical_Depth_047"
start_date = "2004-01-01"
end_date = "2024-02-29"
aoi = ee.Geometry.Point([-78.45, -0.01])

# Crear la colección de imágenes
img_collection = ee.ImageCollection(collection)\
                    .filterBounds(aoi)\
                    .select(band)\
                    .filterDate(start_date, end_date)

# Función para agregar la fecha como una propiedad
def add_date(image):
    return image.set('date', ee.Date(image.date()).format('YYYY-MM-dd'))

# Agregar la fecha como una propiedad a cada imagen
img_collection_with_date = img_collection.map(add_date)

# Obtener la lista de fechas únicas de la colección original
dates_list = img_collection_with_date.aggregate_array('date').distinct()

# Contar el número de imágenes únicas (una por día)
count = dates_list.size().getInfo()

print(f"La colección tiene {count} archivo/s")

La colección tiene 7364 archivo/s


In [ ]:
# Función para extraer el valor del píxel en la geometría 'aoi'
def extract_pixel_value(image):
    value = image.reduceRegion(reducer=ee.Reducer.first(), geometry=aoi, scale=1000).get(band)
    return image.set('pixel_value', value)

# Mapear la función sobre la colección de imágenes
img_collection_with_values = img_collection_with_date.map(extract_pixel_value)

# Crear una lista para almacenar los datos
data = []

# Iterar sobre cada fecha en la lista de fechas
for date in dates_list.getInfo():
    # Filtrar la imagen para la fecha específica
    image_for_date = img_collection_with_values.filterMetadata('date', 'equals', date).first()
    # Obtener el valor del píxel para ese día
    pixel_value_for_date = image_for_date.get('pixel_value').getInfo()
    # Agregar la fecha y el valor del píxel a la lista de datos
    data.append({'date': date, 'pixel_value': pixel_value_for_date})

# Crear un DataFrame de pandas con los datos
df = pd.DataFrame(data)

# Definir la ruta de exportación 
export_path = r'C:\Users\ADMIN\Desktop\TESIS\pixel_values.csv'

# Exportar el DataFrame como un archivo CSV 
df.to_csv(export_path, index=False)

print(f"Archivo CSV exportado exitosamente a: {export_path}")